In [1]:
import os, shutil

username = 'azamahmed'
original_dataset_dir = f'/Users/{username}/dogs-vs-cats/train'

base_dir = '/Users/azamahmed/cats_and_dogs_small'

if not os.path.isdir(base_dir):
  os.mkdir(base_dir) # create the directory for the data

In [2]:
directory_names = ['train', 'validation', 'test']
full_directories = []

for directory in directory_names:
  new_dir = os.path.join(base_dir, directory)
  full_directories.append(new_dir)
  if not os.path.isdir(new_dir):
    os.mkdir(new_dir)

In [3]:
full_directories

['/Users/azamahmed/cats_and_dogs_small/train',
 '/Users/azamahmed/cats_and_dogs_small/validation',
 '/Users/azamahmed/cats_and_dogs_small/test']

In [4]:
all_directories = []

for full_directory in full_directories:
  cats_dir = os.path.join(full_directory, 'cats')
  if not os.path.isdir(cats_dir):
    os.mkdir(cats_dir)
  all_directories.append(cats_dir)

  dogs_dir = os.path.join(full_directory, 'dogs')
  if not os.path.isdir(dogs_dir):
    os.mkdir(dogs_dir)
  all_directories.append(dogs_dir)

In [5]:
all_directories

['/Users/azamahmed/cats_and_dogs_small/train/cats',
 '/Users/azamahmed/cats_and_dogs_small/train/dogs',
 '/Users/azamahmed/cats_and_dogs_small/validation/cats',
 '/Users/azamahmed/cats_and_dogs_small/validation/dogs',
 '/Users/azamahmed/cats_and_dogs_small/test/cats',
 '/Users/azamahmed/cats_and_dogs_small/test/dogs']

In [6]:
fnames = [f'cat.{i}.jpg' for i in range(1000)]
for fname in fnames:
  src = os.path.join(original_dataset_dir, fname)
  dst = os.path.join(all_directories[0], fname)
  shutil.copyfile(src, dst)

fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
  src = os.path.join(original_dataset_dir, fname)
  dst = os.path.join(all_directories[2], fname)
  shutil.copyfile(src, dst)

fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
  src = os.path.join(original_dataset_dir, fname)
  dst = os.path.join(all_directories[4], fname)
  shutil.copyfile(src, dst)

In [7]:
fnames = [f'dog.{i}.jpg' for i in range(1000)]
for fname in fnames:
  src = os.path.join(original_dataset_dir, fname)
  dst = os.path.join(all_directories[1], fname)
  shutil.copyfile(src, dst)

fnames = [f'dog.{i}.jpg' for i in range(1000, 1500)]
for fname in fnames:
  src = os.path.join(original_dataset_dir, fname)
  dst = os.path.join(all_directories[3], fname)
  shutil.copyfile(src, dst)

fnames = [f'dog.{i}.jpg' for i in range(1500, 2000)]
for fname in fnames:
  src = os.path.join(original_dataset_dir, fname)
  dst = os.path.join(all_directories[5], fname)
  shutil.copyfile(src, dst)

In [51]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [47]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 15, 15, 128)      

In [48]:
from tensorflow.keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=['acc'])

In [49]:
from keras.preprocessing.image import ImageDataGenerator

# Turn image file into preprocessed tensors

# Apply data augmentation
train_datagen = ImageDataGenerator(
  rescale=1./255, # Rescale all images by 1/255
  rotation_range=40, # range which to randomly rotate pictures
  width_shift_range=0.2, # randomly translate width
  height_shift_range=0.2, # randomly translate height
  shear_range=0.2, # randomly apply shearing transformations
  zoom_range=0.2, # randomly zoom inside pictures
  horizontal_flip=True, # randomly flipping half the images horizontally
  fill_mode='nearest' # fill new pixels after a transformation
)

test_datagen = ImageDataGenerator(rescale=1./255)

# class_mode is binary because binary_crossentropy is used
train_generator = train_datagen.flow_from_directory(full_directories[0], 
                                                    target_size=(150, 150), 
                                                    batch_size=32, 
                                                    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(full_directories[1],
                                                        target_size=(150, 150),
                                                        batch_size=32,
                                                        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [50]:
# Error
history = model.fit(
      train_generator,
      steps_per_epoch=100,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=50)

TypeError: __array__() takes 1 positional argument but 2 were given